In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.differentiation import FiniteDifference

# Lorenz system
def lorenz(t, state, sigma=10, beta=8/3, rho=28):
    x, y, z = state
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

# Time grid
dt = 0.01
t_end = 32
t_eval = np.arange(0, t_end, dt)

# Integrate
sol = solve_ivp(lorenz, [0, t_end], [1, 1, 1], t_eval=t_eval)
x = sol.y.T
t = sol.t

# Compute derivatives using PySINDy's differentiation
fd = FiniteDifference(order=1)
x_dot = fd._differentiate(x, t=dt)

# SINDy setup
library = ps.PolynomialLibrary(degree=2)
optimizer = ps.STLSQ(threshold=0.1)
model = ps.SINDy(feature_library=library, optimizer=optimizer)

# Fit model with explicit derivatives
model.fit(x, x_dot=x_dot, t=dt, ensemble=True, n_models=30, n_subset=500, quiet=True)
model.print()


AttributeError: module 'numpy' has no attribute 'math'